# Sesión 11: Implementación del sistema RSA

A continuación se muestra:

> Un programa ```RSA_claves``` que calcula las claves pública y privada. El input del programa debe ser el número de cifras de los primos que se van a usar.

> Otro programa ```codificado``` que codifique el mensaje usando la clave pública $(N,e)$.

> Otro programa ```descodificado```que descodifique el mensaje codificado $c$ usando la clave privada $(N,f)$.

Para ello pueden ser útiles las siguientes funciones de sagemath:

```secrets.randbelow(n)``` Devuelve un número aleatorio menor que $n$.

```is_prime(n)``` Devuelve True o False dependiendo de si $n$ es primo o no.

```gcd(a,b)``` Devuelve el máximo común divisor de $a$ y $b$.

```lcm(a,b)``` Devuelve el mínimo común múltiplo de $a$ y $b$.

```ord(m)``` Convierte un caracter en un número siguiendo el sistema ASCII

```chr(n)``` Convierte un número en un caracter siguiendo el sistema ASCII

```IntegerModRing(m)``` El anillo de los enteros módulo $m$.

```m.digits()``` Fabrica una lista con los dígitos de un número natural $m$.


En la práctica, el receptor del mensaje fabricaría las claves con el programa ```RSA_claves``` y enviaría la clave pública al emisor. Este usaría el programa ```codificado``` para codificar el mensaje y enviaría este mensaje codificado $c$ al receptor, que lo descodificaría usando el programa ```descodificado```

Para usar los módulos ```secrets``` e ```integer_mod``` hay que ejecutar la siguiente celda, donde se da algunos ejemplos de cálculo de una potencia en los enteros modulares.







In [1]:
# EJEMPLO DE USO DE LAS LIBRERÍAS
from sage.rings.finite_rings.integer_mod import IntegerMod
import secrets

# La función random no produce números suficientemente aleatorios. 
# Por esta razón utilizaremos secrets.randbelow(n).

# Voy a calcular 4^5 mod 13 y 4^12 mod 13
R = IntegerModRing(13)
print(R(4)^5)
print(R(4)^12)
m = R(4)
print(m^5)
R(4) == R(17)

10
1
10


True

In [25]:
# Programa RSA_Claves
def RSA_claves(n):
    # Calculamos el número p
    p = secrets.randbelow(int(10^n))
    while (is_prime(p) == False):
        p = secrets.randbelow(int(10^n))
    
    # Calculamos el núemero q
    q = secrets.randbelow(int(10^n))
    while (is_prime(q) == False):
        q = secrets.randbelow(int(10^n))
    
    # Ahora calculamos N con p y q
    N = p * q
    
    # Mínimo común múltiplo de p-1 y q-1
    mcm = lcm(p-1, q-1)
    
    # e * f - 1 = r * mcm
    # Primero elijo e coprimo con p-1 y q-1
    e = secrets.randbelow(int(10^n))
    while (gcd(e,p-1)!=1 and gcd(e,q-1)!=1):
        e = secrets.randbelow(int(10^n))
    e = Integer(e)
    
    # Guardamos el valor de E que usaremos luego en la clave pública.
    E = e
    
    # Ahora encuentro f usando el algoritmo de Euclides extendido
    # e * f - r * mcm = 1
    
    # Primeros valores necesarios para el algoritmo.
    cociente = (e - e.mod(mcm))//mcm
    resto = e.mod(mcm)
    e = mcm
    mcm = resto 
    x0 = 0
    x1 = 1
    y0 = 1
    y1 = -cociente
    
    # Bucle en el que hacemos el algoritmo de Euclides
    while resto!=0:
        
        # Calculamos el resto euclídeo.
        cociente = (e - e.mod(mcm))//mcm
        resto = e.mod(mcm)
        
        # Vamos cambiando los valores de a y de b.
        e = mcm
        mcm = resto
        
        # Vamos cambiando los valores de x0, x1, y0, y1.
        x0, x1 = x1, x0 - cociente*x1
        y0, y1 = y1, y0 - cociente*y1
    
    # Valor de f
    f = x0
    
    # Ya tenemos los valores de las claves pública y privada
    clave_publica = [N, E]
    clave_privada = [N, f]
    
    return clave_publica, clave_privada

# Guardamos las claves pública y privada en dos variables globales
clave_publica, clave_privada = RSA_claves(350)
print(f"La clave pública es: {clave_publica}")
print(f"La clave privada es: {clave_privada}")

La clave pública es: [2711381459996342213385954210945290095881149844971867313628959752495034850950642479058425223806325433262907726014644639617904301952294092940561888954063913304179572093154252094604945483336222647256488363553585388590874264605153243699013544681343415607072895061175858886494125804970870678702820750159002293623159764250265442927828046266146229531275326949395523870247553660111363353192826043121596816943768663892838815865293614011849158578469175027237366340864471746539898006624017233913947547588210297080089000366124366881217323372316235261092165437330946542780685400546419409482050380836626982662508365625275295480602416612309498201377151795307647895668087800233600779264797850654176417198207684474903, 236606440718775569870740850193510826509480415185651741330468871861526353052794986671020657709334205975395031938927027514826335278422994888225529931925093038827658067345386630953143533710035511481119219988610442074878103860540772956577530212571677116501004372302405659638227562

In [26]:
# Programa codificado
def codificado(clave_publica):
    # Guardamos las variables N y e de la clave pública
    N = clave_publica[0]
    e = clave_publica[1]
    
    # Trabajamos en módulo N
    R = IntegerModRing(N)
    r = int((log(N,128)/2-1).n())
    
    # Pedimos el mensaje
    print('Introduzca el mensaje a codificar. Ha de tener a lo sumo', r,'caracteres')
    m = input()
    
    # Calculamos el mensaje codificado
    m = R(ZZ([ord(a) for a in list(m)],128)) #ZZ(L,128) halla el entero cuyos dígitos en base 128 forman la lista L.
    c = m^e
    print('El mensaje codificado es',c)
    
    return c
    
# Llamamos a la función
c = codificado(clave_publica)

Introduzca el mensaje a codificar. Ha de tener a lo sumo 164 caracteres


 Me llamo Pablo


El mensaje codificado es 2524101855852789950260161035887050436066244705217973983616219231442207071324294887305722252704418548707583518619660119019672093099394304185977876217091564043895687534151024981383404937585795245663362154767122780890484091928694782925503974000990507096099564358967325828180517869729604873042515918614120417468556414661673751193325675372120379014160249162316961971282371048546280021369764933409530309793075515923273574218548626144514178451701054357716544928929357029122271640918836142557939741341752921150020567725113445322148887623679314397500330152630806967139608110763966922774211189666545577922645776302633957911236159869413500642414629952906531640791087884525954237748888506448701297525630275602271


In [27]:
# Programa descodificado
def descodificado(c, clave_privada):
    # Guardamos las variables N y f de la clave privada
    N = clave_privada[0]
    f = clave_privada[1]
    
    # Trabajamos en módulo N
    R = IntegerModRing(N)
    m = R(c)^f
    
    # Pasamos el mensaje de sistema ASCII a caracteres
    mm = ''.join([chr(a) for a in Integer(m).digits(128)]) #'+'.join(['a','b','c']) = 'a+b+c'
    print('El mensaje descodificado es',mm)
    
    return mm
    
# Llamamos a la función que descodifica el mensaje
mensaje = descodificado(c, clave_privada)  

El mensaje descodificado es Me llamo Pablo


Determina una cota para el número de pasos que se necesitan para calcular el máximo común divisor de dos números de 350 cifras usando el algoritmo de Euclides (este ejercicio se hace)